# Credit EDA Case Study - Workbook
## Submitted By: Alex Anthraper & Amit Haldar

## Bckground


   #### - Cases when company may face the financial loss
    - If the applicant is likely to repay, but not approving the loan
    - If the applicant is not likely to repay, but approving the loan
   #### - Two types of scenarios of the loan applicants
    - The client with payment difficulties: late payment more than X days on at least one of the first  Y instalments of the loan
    - All other cases: All other cases when the payment is paid on time.
   
   #### - When a client applies for a loan, there are four types of decisions that could be taken by the client/company):

    - Approved, Cancelled (by the client due to a higher risk), Refused (rejected by the company), Unused offer (approved but cancelled by the client at the later stage) 
    
## Business Objectives: 

### Analysing the patterns in the consumer and loan application data using EDA for ensuring, whether or not, the applicants are capable of repaying the loans and are not rejected

    - To identify patterns indicating difficulties in paying installments because of:
        - Denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate

    - In other words, the company wants to identify and understand the driving factors (variables) an indicators for possible loan defaults

## Alternate steps:
 Some steps that can be followed to massage the data
 
#### 1. curr - Remove columns with null > 50%
#### 2. prev - Remove columns with null > 50%
#### 3. prev - Aggregate on SK_ID_CURR
#### 4. merge prev and curr
#### 5. examine nulls again and handle them
#### 6. Univariate analysis
#### 7. Bivariate analysis

 --- to be continued


#### Importing required libraries

In [1]:
#import the warnings.
import warnings

warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
pd.set_option('max_rows', 500)
pd.set_option('max_columns', 500)

In [4]:
curr_app0 = pd.read_csv('data/application_data.csv')
prev_app0 = pd.read_csv('data/previous_application.csv')

### 1. Check with the normal routine (head, shae, info, describe etc.)

In [5]:
print(curr_app0.shape)
print(prev_app0.shape)

(307511, 122)
(1670214, 37)


#### Finding the % of null in each column

In [6]:
# df0.isnull().sum()/len(df0) = mean
# Therefore, we can directly use the mean function¶

curr_app_null = curr_app0.isnull().mean().round(4)*100
curr_app_null

SK_ID_CURR                       0.00
TARGET                           0.00
NAME_CONTRACT_TYPE               0.00
CODE_GENDER                      0.00
FLAG_OWN_CAR                     0.00
FLAG_OWN_REALTY                  0.00
CNT_CHILDREN                     0.00
AMT_INCOME_TOTAL                 0.00
AMT_CREDIT                       0.00
AMT_ANNUITY                      0.00
AMT_GOODS_PRICE                  0.09
NAME_TYPE_SUITE                  0.42
NAME_INCOME_TYPE                 0.00
NAME_EDUCATION_TYPE              0.00
NAME_FAMILY_STATUS               0.00
NAME_HOUSING_TYPE                0.00
REGION_POPULATION_RELATIVE       0.00
DAYS_BIRTH                       0.00
DAYS_EMPLOYED                    0.00
DAYS_REGISTRATION                0.00
DAYS_ID_PUBLISH                  0.00
OWN_CAR_AGE                     65.99
FLAG_MOBIL                       0.00
FLAG_EMP_PHONE                   0.00
FLAG_WORK_PHONE                  0.00
FLAG_CONT_MOBILE                 0.00
FLAG_PHONE  

#### Isolating all columns with null% > 50 and putting them in null_cols list

In [7]:
curr_app_nullcols = curr_app_null[curr_app_null > 50]
curr_app_nullcols = list(curr_app_nullcols.index)
curr_app_nullcols

['OWN_CAR_AGE',
 'EXT_SOURCE_1',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BUILD_MEDI',
 'COMMONAREA_MEDI',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'FLOORSMIN_MEDI',
 'LANDAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAREA_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAREA_MEDI',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE']

#### Dropping all columns in the null_cols list and putting in a curr_app1 dataframe

In [8]:
curr_app1 = curr_app0.drop(curr_app_nullcols, axis=1)
print(curr_app0.shape)
print(curr_app1.shape)
curr_app1.describe()

(307511, 122)
(307511, 81)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,FLOORSMAX_MODE,YEARS_BEGINEXPLUATATION_MEDI,FLOORSMAX_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307509.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,3.068510e+05,246546.000000,157504.000000,154491.000000,157504.000000,154491.000000,157504.000000,154491.000000,159080.000000,306490.000000,306490.000000,306490.000000,306490.000000,307510.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.00000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,-4986.120328,-2994.202373,0.999997,0.819889,0.199368,0.998133,0.281066,0.056720,2.152665,2.052463,2.031521,12.063419,0.015144,0.050769,0.040659,0.078173,0.230454,0.179555,5.143927e-01,0.510853,0.977735,0.226282,0.977065,0.222315,0.977752,0.225897,0.102547,1.422245,0.143421,1.405292,0.100049,-962.858788,0.000042,0.710023,0.000081,0.015115,0.088055,0.000192,0.081376,0.003896,0.000023,0.003912,0.000007,0.003525,0.002936,0.00121,0.009928,0.000267,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,3522.886321,1509.450419,0.001803,0.384280,0.399526,0.043164,0.449521,0.231307,0.910682,0.509034,0.502737,3.265832,0.122126,0.219526,0.197499,0.268444,0.421124,0.383817,1.910602e-01,0.194844,0.059223,0.144641,0.064575,0.143709,0.059897,0.145067,0.107462,2.400989,0.446698,2.379803,0.362291,826.808487,0.006502,0.453752,0.009016,0.122010,0.283376,0.013850,0.273412,0.062295,0.004771,0.062424,0.002550,0.059268,0.054110,0.03476,0.099144,0.016327,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,-24672.000000,-7197.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.173617e-08,0.000527,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4292.000000,0.000000,0.000000,0

#### Confirming that SK_ID_CURR is primary key in curr_app1 and foreign key in prev_app0

In [9]:
print(curr_app0.SK_ID_CURR.value_counts().max())
print(prev_app0.SK_ID_CURR.value_counts().max())

1
77


#### Checking out the histories of some of the applications

In [10]:
print(prev_app0.SK_ID_CURR.value_counts())
#[['NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']]
prev_app0[prev_app0.SK_ID_CURR == 100003]

187868    77
265681    73
173680    72
242412    68
206783    67
          ..
382489     1
426056     1
454726     1
380442     1
124145     1
Name: SK_ID_CURR, Length: 338857, dtype: int64


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
575941,1810518,100003,Cash loans,98356.995,900000.0,1035882.0,NaN,900000.0,FRIDAY,12,Y,1,NaN,NaN,NaN,XNA,Approved,-746,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
1021650,2636178,100003,Consumer loans,64567.665,337500.0,348637.5,0.0,337500.0,SUNDAY,17,Y,1,0.000000,NaN,NaN,XAP,Approved,-828,Cash through the bank,XAP,Family,Refreshed,Furniture,POS,XNA,Stone,1400,Furniture,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0
1223745,2396755,100003,Consumer loans,6737.310,68809.5,68053.5,6885.0,68809.5,SATURDAY,15,Y,1,0.100061,NaN,NaN,XAP,Approved,-2341,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,Country-wide,200,Consumer electronics,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0


In [11]:
prev_app0['EMIRatio'] = prev_app0.AMT_ANNUITY / prev_app0.AMT_APPLICATION


In [12]:
prev_app1 = prev_app0.pivot_table(index = 'SK_ID_CURR', columns='NAME_CONTRACT_STATUS', values='SK_ID_PREV', aggfunc='count', fill_value=0).reset_index()
prev_app1.index.name = None
prev_app1

NAME_CONTRACT_STATUS,SK_ID_CURR,Approved,Canceled,Refused,Unused offer
0,100001,1,0,0,0
1,100002,1,0,0,0
2,100003,3,0,0,0
3,100004,1,0,0,0
4,100005,1,1,0,0
...,...,...,...,...,...
338852,456251,1,0,0,0
338853,456252,1,0,0,0
338854,456253,2,0,0,0
338855,456254,2,0,0,0


In [13]:

prev_app_approved = prev_app0[prev_app0.NAME_CONTRACT_STATUS == 'Approved'].groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': 'count',
    'NAME_CONTRACT_TYPE': lambda x:x.value_counts().index[0],
    'EMIRatio': 'median',
    'RATE_INTEREST_PRIMARY': 'median',
    'CNT_PAYMENT': 'median',
    'NAME_YIELD_GROUP': lambda x:x.value_counts().index[0],
    'NFLAG_INSURED_ON_APPROVAL': 'sum'
}).reset_index()
prev_app_approved

,SK_ID_CURR,SK_ID_PREV,NAME_CONTRACT_TYPE,EMIRatio,RATE_INTEREST_PRIMARY,CNT_PAYMENT,NAME_YIELD_GROUP,NFLAG_INSURED_ON_APPROVAL
0,100001,1,Consumer loans,0.159087,NaN,8.0,high,0.0
1,100002,1,Consumer loans,0.051670,NaN,24.0,low_normal,0.0
2,100003,3,Consumer loans,0.109286,NaN,12.0,middle,2.0
3,100004,1,Consumer loans,0.220626,NaN,4.0,middle,0.0
4,100005,1,Consumer loans,0.107877,NaN,12.0,high,0.0
...,...,...,...,...,...,...,...,...
337693,456251,1,Consumer loans,0.163290,NaN,8.0,high,0.0
337694,456252,1,Consumer loans,0.174918,NaN,6.0,low_normal,1.0
337695,456253,2,Consumer loans,0.198616,NaN,5.0,high,1.0
337696,456254,2,Consumer loans,0.103524,NaN,15.0,low_normal,1.0


In [ ]:
prev_app_refused = prev_app0[prev_app0.NAME_CONTRACT_STATUS == 'Refused'].groupby('SK_ID_CURR').agg({
    'NAME_CONTRACT_TYPE': lambda x:x.value_counts().index[0],
    'EMIRatio': 'median',
    'RATE_INTEREST_PRIMARY': 'median',
    'CNT_PAYMENT': 'median',
    'NAME_YIELD_GROUP': lambda x:x.value_counts().index[0],
    'NFLAG_INSURED_ON_APPROVAL': 'sum',
    'CODE_REJECT_REASON': lambda x:x.value_counts().index[0],
}).reset_index()
prev_app_refused

In [ ]:
prev_app_cancelled = prev_app0[prev_app0.NAME_CONTRACT_STATUS == 'Canceled'].groupby('SK_ID_CURR').agg({
    'NAME_CONTRACT_TYPE': lambda x:x.value_counts().index[0],
    'EMIRatio': 'median',
    'RATE_INTEREST_PRIMARY': 'median',
    'CNT_PAYMENT': 'median',
    'NAME_YIELD_GROUP': lambda x:x.value_counts().index[0],
    'NFLAG_INSURED_ON_APPROVAL': 'sum',
    'CODE_REJECT_REASON': lambda x:x.value_counts().index[0],
}).reset_index()
prev_app_cancelled

In [ ]:
prev_app_unused = prev_app0[prev_app0.NAME_CONTRACT_STATUS == 'Unused offer'].groupby('SK_ID_CURR').agg({
    'NAME_CONTRACT_TYPE': lambda x:x.value_counts().index[0],
    'EMIRatio': 'median',
    'RATE_INTEREST_PRIMARY': 'median',
    'CNT_PAYMENT': 'median',
    'NAME_YIELD_GROUP': lambda x:x.value_counts().index[0],
    'NFLAG_INSURED_ON_APPROVAL': 'sum',
    'CODE_REJECT_REASON': lambda x:x.value_counts().index[0],
}).reset_index()
prev_app_unused

In [ ]:
prev_app_approved.rename(columns={'SK_ID_PREV':'Total_Appl', 'NAME_CONTRACT_TYPE': 'Most_Approvals_For', 'EMIRatio':'Approved_EMIRatio', 'RATE_INTEREST_PRIMARY': 'Approved_RATE_INTEREST_PRIMARY', 'CNT_PAYMENT':'Approved_CNT_PAYMENT', 'NAME_YIELD_GROUP':'Approval_NAME_YIELD_GROUP', 'NFLAG_INSURED_ON_APPROVAL':'Approval_insured'}, inplace=True)
prev_app_refused.rename(columns={'NAME_CONTRACT_TYPE': 'Most_Refusals_For', 'EMIRatio':'Refused_EMIRatio', 'RATE_INTEREST_PRIMARY': 'Refused_RATE_INTEREST_PRIMARY', 'CNT_PAYMENT':'Refused_CNT_PAYMENT', 'NAME_YIELD_GROUP':'Refused_NAME_YIELD_GROUP', 'NFLAG_INSURED_ON_APPROVAL':'Refused_Insured', 'CODE_REJECT_REASON':'Refused_CODE_REJECT_REASON'}, inplace=True)
prev_app_cancelled.rename(columns={'NAME_CONTRACT_TYPE': 'Most_Cancels_For', 'EMIRatio':'Cancelled_EMIRatio', 'RATE_INTEREST_PRIMARY': 'Cancelled_RATE_INTEREST_PRIMARY', 'CNT_PAYMENT':'Cancelled_CNT_PAYMENT', 'NAME_YIELD_GROUP':'Cancelled_NAME_YIELD_GROUP', 'NFLAG_INSURED_ON_APPROVAL':'Cancelled_Insured', 'CODE_REJECT_REASON':'Cancelled_CODE_REJECT_REASON'}, inplace=True)
prev_app_unused.rename(columns={'NAME_CONTRACT_TYPE': 'Most_Unused_For', 'EMIRatio':'Unused_EMIRatio', 'RATE_INTEREST_PRIMARY': 'Unused_RATE_INTEREST_PRIMARY', 'CNT_PAYMENT':'Unused_CNT_PAYMENT', 'NAME_YIELD_GROUP':'Unused_NAME_YIELD_GROUP', 'NFLAG_INSURED_ON_APPROVAL':'Unused_insured', 'CODE_REJECT_REASON':'Unused_CODE_REJECT_REASON'}, inplace=True)

In [ ]:
prev_app2 = prev_app1.merge(prev_app_approved, on='SK_ID_CURR', how='left')
prev_app2 = prev_app2.merge(prev_app_refused, on = 'SK_ID_CURR', how='left')
prev_app2 = prev_app2.merge(prev_app_cancelled, on = 'SK_ID_CURR', how='left')
prev_app2 = prev_app2.merge(prev_app_unused, on = 'SK_ID_CURR', how='left')


In [ ]:
prev_app2.to_csv('data/prev_appl_aggregated', index=False)